In [0]:
import zipfile, os
zip_ref = zipfile.ZipFile('mongolian-bert-master.zip', 'r')
zip_ref.extractall('./')
os.chdir('./mongolian-bert-master/')
zip_ref.close()

In [0]:
from google.colab import auth
auth.authenticate_user()

In [3]:
!pip3 install -r requirements.txt
!pip3 install pytorch-pretrained-bert

    100% |████████████████████████████████| 1.0MB 17.6MB/s 
    100% |████████████████████████████████| 81kB 26.8MB/s 
    100% |████████████████████████████████| 112kB 34.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/84/11/01/951369cbbf8f96878786a1f4da68bd7ac19a5d945b38e03d54
Successfully built EbookLib
    100% |████████████████████████████████| 122kB 3.8MB/s 


In [0]:
import torch
from tokenization_sentencepiece import FullTokenizer
from pytorch_pretrained_bert import BertModel, BertForMaskedLM, BertForNextSentencePrediction

In [0]:
import logging
logging.basicConfig(level=logging.INFO)

In [6]:
# Load pre-trained model tokenizer 
tokenizer = FullTokenizer(model_file='model-32k/mn_cased.model', vocab_file='model-32k/mn_cased.vocab', do_lower_case=False)

Loaded a trained SentencePiece model.


In [0]:
text = "Монгол бахархлын өдөр буюу Их эзэн Чингис хааны мэндэлсэн өдөр өчигдөр тохиов Эрдэмтэд Чингис хааны мэндэлсэн өдрийг билгийн тооллын гуравдугаар жарны усан морин жил буюу 1162 оны өвлийн тэргүүн сарын шинийн 1-ний өдөр хэмээн тогтоосон байдаг"
tokenized_text = tokenizer.tokenize(text)

In [8]:
masked_index = 6
tokenized_text[masked_index] = '[MASK]'
print(tokenized_text)

['▁Монгол', '▁бахархлын', '▁өдөр', '▁буюу', '▁Их', '▁эзэн', '[MASK]', '▁хааны', '▁мэндэлсэн', '▁өдөр', '▁өчигдөр', '▁тохио', 'в', '▁Эрдэмтэд', '▁Чингис', '▁хааны', '▁мэндэлсэн', '▁өдрийг', '▁билгийн', '▁тооллын', '▁гуравдугаар', '▁жарны', '▁усан', '▁морин', '▁жил', '▁буюу', '▁11', '62', '▁оны', '▁өвлийн', '▁тэргүүн', '▁сарын', '▁шинийн', '▁1-', 'ний', '▁өдөр', '▁хэмээн', '▁тогтоосон', '▁байдаг']


In [0]:
# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [10]:
len(segments_ids) == len(tokenized_text)

True

In [0]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [0]:
mkdir model

In [17]:
!gsutil cp gs://mongolian-bert/model-32k/model/model.ckpt-952000.meta model/
!gsutil cp gs://mongolian-bert/model-32k/model/model.ckpt-952000.index model/
!gsutil cp gs://mongolian-bert/model-32k/model/model.ckpt-952000.data-00000-of-00001 model/

Copying gs://mongolian-bert/model-32k/model/model.ckpt-952000.meta...
/ [1 files][  4.7 MiB/  4.7 MiB]                                                
Operation completed over 1 objects/4.7 MiB.                                      
Copying gs://mongolian-bert/model-32k/model/model.ckpt-952000.index...
/ [1 files][  9.1 KiB/  9.1 KiB]                                                
Operation completed over 1 objects/9.1 KiB.                                      
Copying gs://mongolian-bert/model-32k/model/model.ckpt-952000.data-00000-of-00001...
- [1 files][  1.2 GiB/  1.2 GiB]   61.1 MiB/s                                   
Operation completed over 1 objects/1.2 GiB.                                      


In [0]:
!cp model-32k/bert_config.json model/

In [38]:
!python3 convert_tf_checkpoint_to_pytorch.py --tf_checkpoint_path model/model.ckpt-952000 --bert_config_file model/bert_config.json --pytorch_dump_path model/pytorch_model.bin

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Converting TensorFlow checkpoint from /content/mongolian-bert-master/model/model.ckpt-952000 with config at /content/mongolian-bert-master/model/bert_config.json
Loading TF weight bert/embeddings/LayerNorm/beta with shape [768]
Loading TF weight bert/embeddings/LayerNorm/beta/adam_m with shape [768]
Loading TF weight bert/embeddings/LayerNorm/beta/adam_v with shape [768]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [768]
Loading TF weight bert/embeddings/LayerNorm/gamma/adam_m with shape [768]
Loading TF weight bert/embeddings/LayerNorm/gamma/adam_v with shape [768]
Loading TF weight bert/embeddings/position_embeddings with shape [512, 768]
Loading TF weight bert/embeddings/position_embeddings/adam_m with shape [512, 768]
Loading TF weight bert/embeddings/position_embeddings/adam_v with shape [512, 768]
Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 768]
Loa

In [39]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('./model/')
model.eval()

INFO:pytorch_pretrained_bert.modeling:loading archive file ./model/
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 32000
}



BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
        )

In [40]:
# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
        )

In [0]:
# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
# We have a hidden states for each of the 12 layers in model bert-base-uncased
assert len(encoded_layers) == 12

In [44]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('./model/')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
    predictions = model(tokens_tensor, segments_tensors)

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
assert predicted_token == '▁Чингис'

INFO:pytorch_pretrained_bert.modeling:loading archive file ./model/
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 32000
}

INFO:pytorch_pretrained_bert.modeling:Weights from pretrained model not used in BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']


In [49]:
predicted_token

'▁Чингис'